In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'data/eng-fra.txt'

# 1 preprocess

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
# orignal text
lines[:10]

['Go.\tVa !',
 'Run!\tCours\u202f!',
 'Run!\tCourez\u202f!',
 'Fire!\tAu feu !',
 "Help!\tÀ l'aide\u202f!",
 'Jump.\tSaute.',
 'Stop!\tÇa suffit\u202f!',
 'Stop!\tStop\u202f!',
 'Stop!\tArrête-toi !',
 'Wait!\tAttends !']

In [8]:
input_texts[:10]

['Go.',
 'Run!',
 'Run!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!']

In [10]:
# "tab" as the "start sequence" character
# "\n" as "end sequence" character
target_texts[:10]

['\tVa !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n']

In [13]:
print(input_characters)
print(target_characters)

{'C', '5', 'o', 't', 'd', 'Q', '6', 'T', 'D', '7', 'p', 'Y', 'z', 'H', 'R', 'I', '0', 'y', 'w', 'k', '?', 'K', 'M', '-', '4', ' ', 'g', 'L', '&', 'N', 'j', 'l', 'e', 'q', 'B', '!', 'm', 'x', ',', 'E', '3', 'r', '%', 'U', 'a', 'P', 'J', 'h', '$', 'c', "'", 'V', '2', 'F', 'A', 'G', '8', 'u', 'f', 'O', 'W', '9', '.', 'n', 'i', 'v', ':', 'S', 's', 'b', '1'}
{'Q', '5', 'D', '0', 'y', 'k', 'g', 'j', 'ê', 'E', '\t', 'ï', 'P', 'J', 'V', '$', 'â', 'F', 'É', 'S', '1', 'o', 'T', 'Y', '’', 'Ê', 'R', '\xa0', 'w', ' ', 'ç', 'e', 'B', 'm', 'x', 'h', "'", '»', 'à', '9', 'v', ':', 's', 'C', 'è', 'î', ')', 'H', 'M', '&', 'l', '«', ',', 'r', 'U', 'ô', 'c', 'ë', 'A', 'G', 'u', '\n', 'f', 'é', '.', 'i', 'n', '\u2009', 'd', 't', '6', 'p', 'z', 'Ç', 'I', '?', 'K', '-', 'ù', 'L', 'œ', 'N', 'q', '!', '3', '\u202f', '%', 'a', '8', 'O', '(', 'û', 'À', 'b'}


In [14]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [15]:
print(input_characters)
print(target_characters)

[' ', '!', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '5', '6', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


In [16]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [17]:
# get token_index dict 
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [19]:
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '$': 2, '%': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '3': 15, '5': 16, '6': 17, '8': 18, '9': 19, ':': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41,

In [20]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [21]:
# Create input data
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [25]:
encoder_input_data[0][0] # in first sample, the char index of time step 0 words

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [32]:
decoder_input_data[0][1]  

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [31]:
decoder_target_data[0][0] # timestep 0 in decoder_target_data is same with timestep 1 in decoder_input_data

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

# 2 model

In [33]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c] # this only output final timestep state

In [34]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [35]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 44s 5ms/step - loss: 0.4926 - val_loss: 0.5964
Epoch 2/10
8000/8000 [==============================] - 43s 5ms/step - loss: 0.4636 - val_loss: 0.5678
Epoch 3/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.4401 - val_loss: 0.5508
Epoch 4/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.4206 - val_loss: 0.5353
Epoch 5/10
8000/8000 [==============================] - 47s 6ms/step - loss: 0.4035 - val_loss: 0.5236
Epoch 6/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.3875 - val_loss: 0.5145
Epoch 7/10
8000/8000 [==============================] - 43s 5ms/step - loss: 0.3731 - val_loss: 0.5014
Epoch 8/10
8000/8000 [==============================] - 42s 5ms/step - loss: 0.3595 - val_loss: 0.4936
Epoch 9/10
8000/8000 [==============================] - 47s 6ms/step - loss: 0.3470 - val_loss: 0.4948
Epoch 10/10
8000/8000 [==

In [38]:
# Save model
model.save('s2s.h5')

/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [41]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  359424      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [54]:
from keras.utils import plot_model 
plot_model(model, show_shapes=True, to_file='seq2seq.png')

In [40]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [44]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


# 3 inference

In [45]:
encoder_model = Model(encoder_inputs, encoder_states)

In [46]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [55]:
decoder_inputs

<tf.Tensor 'input_2:0' shape=(?, ?, 94) dtype=float32>

In [56]:
decoder_states_inputs # this is a list

[<tf.Tensor 'input_3:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 256) dtype=float32>]

In [58]:
[decoder_inputs] + decoder_states_inputs

[<tf.Tensor 'input_2:0' shape=(?, ?, 94) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 256) dtype=float32>]

In [59]:
[[1, 2, 3]] + [[4, 5, 6], [7, 8, 9]]

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [47]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [49]:
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_token_index['\t']] = 1.
target_seq

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [50]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [52]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va chercher !

-
Input sentence: Run!
Decoded sentence: Arrêtez de manger.

-
Input sentence: Run!
Decoded sentence: Arrêtez de manger.

-
Input sentence: Fire!
Decoded sentence: Allez chercher !

-
Input sentence: Help!
Decoded sentence: Arrêtez de me chanter.

-
Input sentence: Jump.
Decoded sentence: Soyez prudente !

-
Input sentence: Stop!
Decoded sentence: Arrêtez de manger.

-
Input sentence: Stop!
Decoded sentence: Arrêtez de manger.

-
Input sentence: Stop!
Decoded sentence: Arrêtez de manger.

-
Input sentence: Wait!
Decoded sentence: Attendez !



# 4 no teacher forcing 

In [60]:
from keras.layers import Lambda
from keras import backend as K

# The first part is unchanged
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = [state_h, state_c]

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Input(shape=(1, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

all_outputs = []
inputs = decoder_inputs
for _ in range(max_decoder_seq_length):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    outputs = decoder_dense(outputs)
    # Store the current prediction (we will concatenate all predictions later)
    all_outputs.append(outputs)
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    inputs = outputs
    states = [state_h, state_c]

# Concatenate all predictions
decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

# Define and compile model as previously
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [62]:
all_outputs

[<tf.Tensor 'dense_2/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_1/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_2/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_3/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_4/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_5/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_6/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_7/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_8/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_9/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_10/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_11/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_12/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_13/truediv:0' shape=(?, 1, 94) dtype=float32>,
 <tf.Tensor 'dense_2_14/truediv:0' shape=(?, 1, 94) dtype=fl

In [61]:
decoder_outputs

<tf.Tensor 'lambda_1/concat:0' shape=(?, 59, 94) dtype=float32>

In [68]:
# Prepare decoder input data that just contains the start character
# Note that we could have made it a constant hard-coded in the model
decoder_input_data = np.zeros((num_samples, 1, num_decoder_tokens))
decoder_input_data[:, 0, target_token_index['\t']] = 1.

# Train model as previously
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 54s 7ms/step - loss: 0.9508 - val_loss: 1.0785
Epoch 2/10
8000/8000 [==============================] - 62s 8ms/step - loss: 0.9021 - val_loss: 1.0603
Epoch 3/10
8000/8000 [==============================] - 61s 8ms/step - loss: 0.8805 - val_loss: 1.0523
Epoch 4/10
8000/8000 [==============================] - 48s 6ms/step - loss: 0.8672 - val_loss: 1.0190
Epoch 5/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.8557 - val_loss: 1.0224
Epoch 6/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.8407 - val_loss: 0.9955
Epoch 7/10
8000/8000 [==============================] - 47s 6ms/step - loss: 0.8247 - val_loss: 0.9833
Epoch 8/10
8000/8000 [==============================] - 49s 6ms/step - loss: 0.8104 - val_loss: 0.9748
Epoch 9/10
8000/8000 [==============================] - 48s 6ms/step - loss: 0.7992 - val_loss: 0.9665
Epoch 10/10
8000/8000 [==